### Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_privacy
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
%matplotlib inline
tf.get_logger().setLevel('ERROR')
tf.compat.v1.disable_v2_behavior()

### Load Dataset

In [2]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 10)
testing: (10000, 32, 32, 3) (10000, 10)


### Model1: Simple CNN

In [3]:
# Hyper-parameters
epochs = 3
batch_size = 512

l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 256
learning_rate = 0.25

if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [4]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [5]:
# define optimizer (dp-sgd) 
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)
# define loss function
loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [6]:
# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# Train model
model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples
Metal device set to: Apple M1 Pro


2022-03-08 12:25:25.078069: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-08 12:25:25.078200: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-08 12:25:25.463276: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-08 12:25:25.470267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 12:25:25.891252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/3


2022-03-08 12:25:26.108440: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-08 12:25:26.190968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11776/50000 [======>.......................] - ETA: 10:32 - loss: 2.2827 - acc: 0.1390

KeyboardInterrupt: 

In [23]:
# Calculate privacy performance
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=x_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta=1e-5)

DP-SGD with sampling rate = 1.02% and noise_multiplier = 1.3 iterated over 293 steps satisfies differential privacy with eps = 0.811 and delta = 1e-05.
The optimal RDP order is 14.0.


(0.81148396243264, 14.0)

### Model2: ResNet with transfer learning

### Model3: HOG classification